In [1]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

def build_generator():

    noise_shape = (100,) 

    model = Sequential()

    model.add(Dense(256,input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    
    return model

In [3]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
       
    return model

In [4]:
def train(epochs, batch_size=128, save_interval=50):

    (train_data, _), (_, _) = mnist.load_data()
    train_data = (train_data.astype(np.float32) - 127.5) / 127.5
    train_data = train_data.reshape(train_data.shape[0],train_data.shape[1],train_data.shape[2],1)
    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):

        idx = np.random.randint(0, train_data.shape[0], half_batch)
        imgs = train_data[idx]
 
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)
        
        discriminator.trainable = True
        
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

        gan_train_noise = np.random.normal(0, 1, (batch_size, 100)) 
        gan_train_target = np.ones((batch_size, 1)) #fake but label-1
        
        discriminator.trainable = False
        
        g_loss = GAN.train_on_batch(gan_train_noise, gan_train_target)
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        if epoch % save_interval == 0:
            save_imgs(epoch)

In [5]:
import numpy as np

idx = np.random.randint(0, 100, 50)
print(idx)

[55 90 88 45 16 81 99 61 95 58 81  2 63 71  5 82  5 63  3 95 87  6 38 10
 65 11 68 38 37 34 87  5 43 25 95 71 86 66 27 49 75 36 82 76 41 19 98 43
 52 56]


In [6]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [7]:
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)
 
noise_input = Input(shape=(100,))   #Our random input to the generator
generator_output = generator(noise_input)
    
discriminator.trainable = False  
  
discriminator_output = discriminator(generator_output)  #Validity check on the generated image

GAN = Model(noise_input, discriminator_output)
GAN.compile(loss='binary_crossentropy', optimizer=optimizer)

train(epochs=10000, batch_size=32, save_interval=10)

generator.save('generator_model.h5')  #Test the model on GAN4_predict...

In [8]:
generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_5 (Dense)              (None, 1024)             

In [9]:
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
__________________________________________________

In [10]:
GAN.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 28, 28, 1)         1493520   
_________________________________________________________________
sequential (Sequential)      (None, 1)                 533505    
Total params: 2,027,025
Trainable params: 1,489,936
Non-trainable params: 537,089
_________________________________________________________________


In [11]:
discriminator.trainable = True
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
__________________________________________________

In [12]:
GAN.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 28, 28, 1)         1493520   
_________________________________________________________________
sequential (Sequential)      (None, 1)                 533505    
Total params: 2,027,025
Trainable params: 2,023,441
Non-trainable params: 3,584
_________________________________________________________________
